# SAMPIC Waveform Visualization

Comprehensive notebook for analyzing SAMPIC waveforms using PyROOT.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
import os

# Set up plotting defaults
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Paths
DATA_FILE = "../output.root"
BUILD_LIB_PATH = "../build/lib"

## Load Libraries and Data

In [ ]:
# Load SAMPIC data product libraries
libs_to_load = [
    "libanalysis_pipeline_core.so",
    "libunpacker_data_products_core.so",
    "libunpacker_data_products_sampic.so",
]

for lib in libs_to_load:
    path = os.path.join(BUILD_LIB_PATH, lib)
    if os.path.exists(path):
        print(f"Loading {lib}...")
        ROOT.gSystem.Load(path)
    else:
        print(f"Warning: {lib} not found")

In [ ]:
# Open the ROOT file
f = ROOT.TFile.Open(DATA_FILE)
if not f or f.IsZombie():
    print(f"Error: Could not open file {DATA_FILE}")
else:
    tree = f.Get("events")
    print(f"Number of events in file: {tree.GetEntries()}")
    print(f"\nAvailable branches:")
    for branch in tree.GetListOfBranches():
        print(f"  - {branch.GetName()}")

## Explore Data Structure

In [ ]:
# Examine first event structure
tree.GetEntry(0)
event = tree.sampic_event

print(f"Number of hits in first event: {len(event.hits)}")
if len(event.hits) > 0:
    hit = event.hits[0]
    print(f"\nFirst hit details:")
    print(f"  Channel: {hit.channel}")
    print(f"  Waveform length: {len(hit.corrected_waveform)}")
    print(f"  Amplitude: {hit.amplitude:.2f}")
    print(f"  Baseline: {hit.baseline:.2f}")
    print(f"  Peak: {hit.peak:.2f}")
    print(f"  Time: {hit.time_instant:.2f} ns")
    print(f"  TOT: {hit.tot_value:.2f} ns")

## Individual Waveform Plots

In [ ]:
def plot_waveform(event_idx, hit_idx):
    """Plot a single waveform from a specific event and hit."""
    tree.GetEntry(event_idx)
    event = tree.sampic_event
    
    if hit_idx >= len(event.hits):
        print(f"Hit {hit_idx} out of range")
        return
    
    hit = event.hits[hit_idx]
    wf = hit.corrected_waveform
    waveform = np.array([wf[i] for i in range(len(wf))], dtype=np.float32)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Plot waveform
    sample_indices = np.arange(len(waveform))
    ax.plot(sample_indices, waveform, 'b-', linewidth=1.5, label='Corrected waveform')
    
    # Mark baseline
    ax.axhline(y=hit.baseline, color='g', linestyle='--', linewidth=1, label=f'Baseline: {hit.baseline:.2f}')
    
    # Mark peak
    ax.axhline(y=hit.peak, color='r', linestyle='--', linewidth=1, label=f'Peak: {hit.peak:.2f}')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Amplitude [ADC]')
    ax.set_title(f'Event {event_idx}, Hit {hit_idx} - Channel {hit.channel}\n'
                 f'Amplitude: {hit.amplitude:.2f}, Time: {hit.time_instant:.2f} ns')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    return fig

# Plot first waveform from first event
tree.GetEntry(0)
if len(tree.sampic_event.hits) > 0:
    plot_waveform(0, 0)
    plt.show()

## Multi-Hit Event Visualization

In [ ]:
def plot_event_waveforms(event_idx, max_hits=4):
    """Plot multiple waveforms from a single event."""
    tree.GetEntry(event_idx)
    event = tree.sampic_event
    num_hits = min(len(event.hits), max_hits)
    
    if num_hits == 0:
        print(f"Event {event_idx} has no hits")
        return None
    
    fig, axes = plt.subplots(num_hits, 1, figsize=(12, 4*num_hits), sharex=True)
    if num_hits == 1:
        axes = [axes]
    
    for i in range(num_hits):
        hit = event.hits[i]
        wf = hit.corrected_waveform
        waveform = np.array([wf[j] for j in range(len(wf))], dtype=np.float32)
        sample_indices = np.arange(len(waveform))
        
        axes[i].plot(sample_indices, waveform, 'b-', linewidth=1.5)
        axes[i].axhline(y=hit.baseline, color='g', linestyle='--', linewidth=1, alpha=0.5)
        axes[i].axhline(y=hit.peak, color='r', linestyle='--', linewidth=1, alpha=0.5)
        
        axes[i].set_ylabel('Amplitude [ADC]')
        axes[i].set_title(f'Hit {i} - Ch {hit.channel} | Amp: {hit.amplitude:.1f} | Time: {hit.time_instant:.2f} ns')
        axes[i].grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Sample Index')
    fig.suptitle(f'Event {event_idx} - All Hits', fontsize=14, fontweight='bold', y=1.001)
    plt.tight_layout()
    
    return fig

# Plot first event with multiple hits
plot_event_waveforms(0, max_hits=4)
plt.show()

## Statistical Analysis

In [ ]:
# Collect data from many events
n_events = min(1000, tree.GetEntries())

amplitudes = []
baselines = []
peaks = []
channels = []
tot_values = []
time_instants = []

for i in range(n_events):
    tree.GetEntry(i)
    event = tree.sampic_event
    for hit in event.hits:
        amplitudes.append(hit.amplitude)
        baselines.append(hit.baseline)
        peaks.append(hit.peak)
        channels.append(hit.channel)
        tot_values.append(hit.tot_value)
        time_instants.append(hit.time_instant)

# Convert to numpy arrays
amplitudes = np.array(amplitudes)
baselines = np.array(baselines)
peaks = np.array(peaks)
channels = np.array(channels)
tot_values = np.array(tot_values)
time_instants = np.array(time_instants)

print(f"Total hits analyzed: {len(amplitudes)}")
print(f"\nAmplitude statistics:")
print(f"  Mean: {np.mean(amplitudes):.2f}")
print(f"  Std:  {np.std(amplitudes):.2f}")
print(f"  Min:  {np.min(amplitudes):.2f}")
print(f"  Max:  {np.max(amplitudes):.2f}")

## Distribution Plots

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Amplitude distribution
axes[0, 0].hist(amplitudes, bins=100, alpha=0.7, edgecolor='black')
axes[0, 0].set_xlabel('Amplitude [ADC]')
axes[0, 0].set_ylabel('Counts')
axes[0, 0].set_title('Amplitude Distribution')
axes[0, 0].set_yscale('log')

# Baseline distribution
axes[0, 1].hist(baselines, bins=100, alpha=0.7, edgecolor='black', color='green')
axes[0, 1].set_xlabel('Baseline [ADC]')
axes[0, 1].set_ylabel('Counts')
axes[0, 1].set_title('Baseline Distribution')

# Channel occupancy
axes[1, 0].hist(channels, bins=64, range=(0, 64), alpha=0.7, edgecolor='black', color='orange')
axes[1, 0].set_xlabel('Channel')
axes[1, 0].set_ylabel('Hits')
axes[1, 0].set_title('Channel Occupancy')

# TOT distribution
axes[1, 1].hist(tot_values, bins=100, alpha=0.7, edgecolor='black', color='purple')
axes[1, 1].set_xlabel('Time over Threshold [ns]')
axes[1, 1].set_ylabel('Counts')
axes[1, 1].set_title('TOT Distribution')
axes[1, 1].set_yscale('log')

plt.tight_layout()
plt.show()

## Peak vs Amplitude Correlation

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))

# Create 2D histogram
h = ax.hist2d(amplitudes, peaks, bins=100, cmap='viridis', 
              cmin=1, norm=plt.matplotlib.colors.LogNorm())
plt.colorbar(h[3], ax=ax, label='Counts')

ax.set_xlabel('Amplitude [ADC]')
ax.set_ylabel('Peak [ADC]')
ax.set_title('Peak vs Amplitude Correlation')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Average Waveform Shape

In [ ]:
# Collect waveforms from first N events
n_events_wf = min(100, tree.GetEntries())
all_waveforms = []

for i in range(n_events_wf):
    tree.GetEntry(i)
    event = tree.sampic_event
    for hit in event.hits:
        wf = hit.corrected_waveform
        if len(wf) > 0:
            wf_array = np.array([wf[j] for j in range(len(wf))], dtype=np.float32)
            # Subtract baseline
            all_waveforms.append(wf_array - hit.baseline)

if len(all_waveforms) > 0:
    # Find maximum length
    max_len = max(len(wf) for wf in all_waveforms)
    
    # Pad shorter waveforms with zeros
    padded_waveforms = np.array([np.pad(wf, (0, max_len - len(wf)), constant_values=0) 
                                  for wf in all_waveforms])
    
    # Compute average and std
    avg_waveform = np.mean(padded_waveforms, axis=0)
    std_waveform = np.std(padded_waveforms, axis=0)
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    sample_indices = np.arange(len(avg_waveform))
    ax.plot(sample_indices, avg_waveform, 'b-', linewidth=2, label='Average')
    ax.fill_between(sample_indices, 
                     avg_waveform - std_waveform, 
                     avg_waveform + std_waveform,
                     alpha=0.3, label='±1 σ')
    
    ax.set_xlabel('Sample Index')
    ax.set_ylabel('Amplitude (baseline subtracted) [ADC]')
    ax.set_title(f'Average Waveform Shape ({len(all_waveforms)} hits)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Analyzed {len(all_waveforms)} waveforms")
    print(f"Maximum waveform length: {max_len} samples")
else:
    print("No waveforms found in the data")